## Initiate the Toolbox
- the parameters set here will be used to collect command-specific data when the `Fetcher` executes the TET pattern.

In [1]:
%load_ext autoreload
%autoreload

from humbldata.toolbox.toolbox_controller import Toolbox

toolbox = Toolbox(
    symbols=["DKNG", "HII", "XYL", "GTBIF", "EAT", "MPW"],
    interval="1d",
    start_date="2000-01-01",
    end_date="2024-02-26",
    membership="admin"
)
toolbox

Toolbox(symbols=['DKNG', 'HII', 'XYL', 'GTBIF', 'EAT', 'MPW'], interval=1d, start_date=2000-01-01, end_date=2024-02-26, provider=yfinance, membership=admin)

## Run Mandelbrot Channel Calculation

In [2]:
# import nest_asyncio
# nest_asyncio.apply()

result = toolbox.technical.mandelbrot_channel(historical=False, window="3m", _boundary_group_down=False, chart=False)
# result.to_polars()

INFO: MandelbrotChannelFetcher || START: fetch_data (sync)
INFO: MandelbrotChannelFetcher || END: fetch_data (sync) - Total time: 4.3744s


In [3]:
json_result_chart = result.to_json(chart=False)

In [12]:
result.results

AttributeError: 'bytes' object has no attribute 'to_polars'

In [4]:
json_result_chart

'{"date": ["2024-02-26", "2024-02-26", "2024-02-26", "2024-02-26", "2024-02-26", "2024-02-26"], "symbol": ["DKNG", "EAT", "GTBIF", "HII", "MPW", "XYL"], "bottom_price": [34.4697, 43.5146, 10.8649, 264.9333, 3.3764, 119.8499], "recent_price": [40.349998474121094, 46.20000076293945, 13.649999618530273, 285.5196533203125, 3.561779737472534, 125.04960632324219], "top_price": [43.3575, 51.0035, 15.6322, 297.413, 3.7028, 127.7862]}'

In [5]:
json_result_chart[0].__class__

str

In [6]:
    import json
    import pytest

    for chart in json_result_chart:
        try:
            json.loads(chart)
        except json.JSONDecodeError:
            pytest.fail(f"Chart data result is not valid JSON: {chart}")

Failed: Chart data result is not valid JSON: {

In [21]:
if isinstance(result.chart, list):
    chart_content = [
        chart.content
        for chart in result.chart
        if chart and chart.content
    ]

In [ ]:
isinstance(chart_content, list)

In [ ]:
data = result.to_polars()
data

In [6]:
from humbldata.toolbox.technical.mandelbrot_channel.model import calc_mandelbrot_channel_historical_mp, calc_mandelbrot_channel_historical_concurrent


# result_mp = calc_mandelbrot_channel_historical_mp(
#     data=data,
#     window="1m",
#     rv_adjustment=True,
#     rv_method="std",
#     rs_method="RS",
#     rv_grouped_mean=False,
#     live_price=False
# )
result_cc = calc_mandelbrot_channel_historical_concurrent(
    data=data,
    window="1m",
    rv_adjustment=True,
    rv_method="std",
    rs_method="RS",
    rv_grouped_mean=False,
    live_price=False,
    use_processes=False,
    max_workers=16
)

In [ ]:
# result_mp.collect()
result_cc.collect()